# Imports, Initializations

In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from item_map import EldenRingItemMap
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
import os
from itertools import batched

# Process data, establish vector database

Specify the items types and fields we want to include in our output documents

In [2]:
item_groups = {
    "Accessory": ["Name", "Caption", "Info"],
    "Arts": ["Name", "Caption"],
    "Gem": ["Name", "Caption", "Effect", "Info"],
    "Goods": ["Name", "Caption", "Effect", "Info", "Info2", "Dialog"],
    "Magic": ["Name", "Caption", "Info"],
    "Protector": ["Name", "Caption", "Info"],
    "Weapon": ["Name", "Caption", "Effect", "Info"]
}
path = "Carian-Archive-main/GameText/GR/data/INTERROOT_win64/msg/engUS/"

In [6]:
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

In [7]:
item_map = EldenRingItemMap(item_groups, path)

# Langchain stuff

In [26]:
def dump_item_to_document(item_group: str, item_id: str, item: dict):
    metadata: dict = {'item_type': item_group}
    item_text: str = f'''Data dump from an item the player can find in the game Elden Ring:
==============================
Item Type: {item_group}
--------------------------
'''
    for field_type, text in item.items():
        item_text += f'''{field_type}: {text}'''
        item_text += "\n--------------------------\n"
        metadata[field_type] = text
    item_doc = {'id': (item_group + "_" + item_id), 'metadata':metadata, 'text_to_embed': item_text}
    return item_doc

In [27]:
simplest_items_as_doc = item_map.dump_items(dump_item_to_document)

In [23]:
item_map.items['Accessory']["3000"]

{'Name': 'Graven-School Talisman',
 'Caption': 'A talisman depicting a school of graven mages, the nightmare of the academy.\n\nRaises potency of sorceries.\n\nThe primeval current is a forbidden tradition of glintstone sorcery. To those who cleave to its teachings, the act of collecting sorcerers to fashion them into the seeds of stars is but another path of scientific inquiry.',
 'Info': 'Raises potency of sorceries'}

In [10]:
len(item_map._blacklist_ids)

731

In [13]:
item_map._blacklist_ids[500]

('Goods', '240010')

In [29]:
embeddings_model = OpenAIEmbeddings(model='text-embedding-3-large')

In [31]:
def add_embeddings(input_docs, embeddings_model):
    embeddings = embeddings_model.embed_documents([doc['text_to_embed'] for doc in input_docs])
    for embedding, doc in zip(embeddings, input_docs):
        doc['values'] = embedding
    return input_docs

In [32]:
simplest_items_as_doc = add_embeddings(simplest_items_as_doc, embeddings_model)

In [37]:
db_commands = []
for item in simplest_items_as_doc:
    command = item.copy()
    del command['text_to_embed']
    db_commands.append(command)

In [38]:
index = pc.Index('elden-ring-default-index')
for batch in batched(db_commands, 100):
    index.upsert(batch)

In [19]:
help(PineconeVectorStore.from_documents)

Help on method from_documents in module langchain_core.vectorstores.base:

from_documents(documents: 'List[Document]', embedding: 'Embeddings', **kwargs: 'Any') -> 'VST' method of abc.ABCMeta instance
    Return VectorStore initialized from documents and embeddings.

    Args:
        documents: List of Documents to add to the vectorstore.
        embedding: Embedding function to use.



In [15]:
db = PineconeVectorStore.from_documents(simplest_items_as_doc, embeddings_model, index_name='elden-ring-default-index')

In [18]:
#db = FAISS.from_documents(simplest_items_as_doc, embeddings_model)
#db.distance_strategy = 'cosine'
#db.save_local('simplest_items_db')

# Testing Search Prototype

In [16]:
def search(query, k=4):
    preamble_string = f'''Answer the following question about the lore of the game Elden Ring, using information provided from a data dump of the game's item text.
    Pay particular attention to the Caption field, if there is one, as this often contains the most lore.
    
    Question:
    '''
    return db.similarity_search_with_score(preamble_string + query, k)

In [17]:
def search_and_print(query, k=4):
    docs = search(query, k)
    for doc, score in docs:
        print(f"score: {score}")
        print("id: ")
        print(doc.id)
        print("doc:")
        print(doc.page_content)

In [18]:
search_and_print("What is the primeval current?", k=20)

score: 0.496391952
id: 
None
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Name: Dragon King's Cragblade
--------------------------
Caption: Piercing Gravel Stone sword containing primeval lightning.
A portion of the Dragonlord's power, gained from a remembrance.

This weapon commands great power over the paltry, mortal dragons of today.
--------------------------

score: 0.495654702
id: 
None
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Founding Rain of Stars
--------------------------
Caption: The eldest primeval sorcery, said to have been discovered by an ancient astrologer. A sorcery of legendary status.

Summons a dark cloud of stars overhead. Shortly after, the cloud will release a violent deluge of star rain. This sorcery can be cast while in motion. Charging increases potency.

Thought to be the founding glintstone sorcery. The glim

In [23]:
search_and_print("Where does glintstone come from?", k=20)

score: 0.5680167078971863
id: 
None
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Sellen's Primal Glintstone
--------------------------
Caption: Glintstone from within the sorceress Sellen's body.
Seemingly half-alive, blood vessels are visible within.

In essence, a primal glintstone is a sorcerer's soul.
If transplanted into a compatible new body, Sellen will rise again.

"My apprentice, do you think it distasteful?"
--------------------------
Info: Glintstone from within Sellen's body
--------------------------

score: 0.5838795900344849
id: 
None
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Glintstone Firefly
--------------------------
Info2: Widely found in Liurnia Lake
--------------------------
Caption: A firefly whose glowing tail has hardened into
glintstone, widely found in Liurnia Lake.
Material used for crafting items.

Ala

In [19]:
search_and_print("Where does glintstone come from?", k=20)

score: 0.5499815940856934
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Sellen's Primal Glintstone
--------------------------
Info: Glintstone from within Sellen's body
--------------------------

score: 0.5892117619514465
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Glintstone Firefly
--------------------------
Info2: Widely found in Liurnia Lake
--------------------------
Info: Material used for crafting items
--------------------------

score: 0.6123149394989014
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Glintstone Scrap
--------------------------
Info: Break gem and use FP to produce a magic bolt
--------------------------

score: 0.6203001737594604
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
-----------------

In [21]:
search_and_print("Why is Ranni a demigod?", k=20)

score: 0.8971127867698669
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Miniature Ranni
--------------------------
Info: Doll resembling Ranni the Witch
--------------------------

score: 0.8980117440223694
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Miniature Ranni
--------------------------
Info: Doll resembling Ranni the Witch
--------------------------

score: 0.9856199026107788
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Ranni's Dark Moon
--------------------------
Info: Incarnate a cold, dark moon and launch it at foes
--------------------------

score: 1.0870332717895508
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Rennala's Full Moon
--------------------------
Info: Incarnat

In [ ]:
search_and_print("Why is Ranni a demigod?", k=20)

score: 0.32297423481941223
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Miniature Ranni
--------------------------
Caption: A doll resembling Ranni the Witch.
From head to toe every detail is perfect.

The chilliness is gone, feeling now like an empty husk.

There is no response.
--------------------------
Info: Doll resembling Ranni the Witch
--------------------------

score: 0.3233301639556885
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Miniature Ranni
--------------------------
Caption: A doll resembling Ranni the Witch.
From head to toe every detail is perfect.

This unresponsive doll seems pleasantly cool.
--------------------------
Info: Doll resembling Ranni the Witch
--------------------------

score: 0.330355703830719
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
------------------

In [ ]:
search_and_print("Where did the Crystalians come from, and what are they, exactly?", k=20)

score: 0.300118088722229
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Name: Crystal Sword
--------------------------
Caption: Sword fashioned from pure crystal; a deed impossible for a human. Enwreathed with powerful magic, its attack scales with intelligence. The inscrutable Crystalians have but one clear purpose; to safeguard their crystals unto the end. One theory posits that they yearn for the return of their creator who will carve for them new brethren.
--------------------------

score: 0.305898517370224
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Name: Crystal Spear
--------------------------
Caption: Spear fashioned from pure crystal; a deed impossible for a human. Enwreathed with powerful magic, its attack scales with intelligence. The inscrutable Crystalians have but one clear purpose; to safeguard their crystals unto the end. One th

In [ ]:
search_and_print("What is the significance of the name 'Elden Ring'?", k=20)

score: 0.3213297724723816
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: The Ring
--------------------------

score: 0.3213297724723816
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: The Ring
--------------------------

score: 0.3376530408859253
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Name: Ringed Finger
--------------------------
Caption: Bludgeon made of an enormous finger sheathed in several heavy rings. Thought to have been cut from an ancestor of the Fingercreeper.

Some life yet remains in this legacy of an ancient act of blasphemy, as evidenced by the barely perceptible warmth it still exudes.
--------------------------

score: 0.3411431908607483
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
---------------------

In [ ]:
search_and_print("Origin of the ants in the eternal cities", k=20)

score: 0.3463672399520874
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Name: Ant's Skull Plate
--------------------------
Caption: Huge head of one of the giant ants which inhabit the two underground rivers, used without modification as a shield. Excels at repelling enemy attacks.

Giant ants are venomous creatures, granting a boost to immunity when wielding this shield.
--------------------------

score: 0.36932411789894104
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Goods
--------------------------
Name: Formic Rock
--------------------------
Info2: Found near Ainsel River and other places where giant ants live
--------------------------
Caption: Rock formed from solidified giant ant venom. Highly acidic. Material used for crafting items.

Found near Ainsel River and other places where giant ants live.
--------------------------
Info: Material used for crafting items
-------

In [ ]:
search_and_print("Who's the most powerful character?", k=20)

score: 0.3841433823108673
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Effect: Attack power rises as attacks continue
--------------------------

score: 0.39214813709259033
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Weapon
--------------------------
Name: Crystal Sword
--------------------------
Caption: Sword fashioned from pure crystal; a deed impossible for a human. Enwreathed with powerful magic, its attack scales with intelligence. The inscrutable Crystalians have but one clear purpose; to safeguard their crystals unto the end. One theory posits that they yearn for the return of their creator who will carve for them new brethren.
--------------------------

score: 0.3925646245479584
doc:
Data dump from an item the player can find in the game Elden Ring:
Item Type: Protector
--------------------------
Info: Armor of Godfrey, the first Elden Lord
--------------------------